In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

## 输入数据处理

In [ ]:
# 所有的类别
class_names = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck',
]

# 路径
train_lables_file = '../cifar10/trainLabels.csv'  # labels
test_csv_file = '../cifar10/sampleSubmission.csv'  # submission
train_folder = '../cifar10/train/'  # 训练集所在的文件夹
test_folder = '../cifar10/test/'  # 测试集所在的文件夹

# def parse_csv_file(filepath, folder):
#     """Parses csv files into (filename(path), label) format"""
#     results = []
#     # 读取所有行
#     with open(filepath, 'r') as f:
# #         lines = f.readlines()  为什么加[1:]？ 有标题head
#         lines = f.readlines()[1:]
#     for line in lines:
#         image_id, label_str = line.strip('\n').split(',')
#         image_full_path = os.path.join(folder, image_id + '.png')
#         results.append((image_full_path, label_str))
#     return results

# # 解析对应的文件夹
# train_labels_info = parse_csv_file(train_lables_file, train_folder)
# test_csv_info = parse_csv_file(test_csv_file, test_folder)

# # 打印
# import pprint
# pprint.pprint(train_labels_info[0:5])
# pprint.pprint(test_csv_info[0:5])
# print(len(train_labels_info), len(test_csv_info))

# 读取 label
train_path_df = pd.read_csv(train_labels_file)
train_path_df["id"] = train_path_df["id"].apply(lambda x: "{}/{}.png".format(train_folder, x))
train_path_df.columns=["filepath", "class"]
train_path_df.head()

test_df = pd.read_csv(test_csv_file)
test_df["path"] = test_df["id"].apply(lambda x: "{}/{}.png".format(test_folder, x))
test_df.head()

In [ ]:
# # train_df = pd.DataFrame(train_labels_info)
# train_df = pd.DataFrame(train_labels_info[0:45000])
# valid_df = pd.DataFrame(train_labels_info[45000:])
# test_df = pd.DataFrame(test_csv_info)

# train_df.columns = ['filepath', 'class']
# valid_df.columns = ['filepath', 'class']
# test_df.columns = ['filepath', 'class']

# print(train_df.head())
# print(valid_df.head())
# print(test_df.head())

In [ ]:
# 数据集划分
train_df = pd.DataFrame(train_path_df[0:45000])
valid_df = pd.DataFrame(train_path_df[45000:])

## 模型搭建

In [ ]:
height = 224
width = 224
channels = 3
batch_size = 24
num_classes = 10

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest',
)
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory = './',
    x_col = 'filepath',
    y_col = 'class',
    # 将labels转换为id
    classes = class_names,
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = True,
    class_mode = 'sparse',
)

# 和上面一致
valid_datagen = keras.preprocessing.image.ImageDataGenerator(
   preprocessing_function = keras.applications.resnet50.preprocess_input)
valid_generator = valid_datagen.flow_from_dataframe(
    valid_df,
    directory = './',
    x_col = 'filepath',
    y_col = 'class',
    classes = class_names,
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    class_mode = "sparse")

train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num, valid_num)

In [ ]:
model = keras.models.Sequential([
    keras.applications.ResNet50(include_top=False, pooling='avg', weights='imagenet'),
    keras.layers.Dense(num_classes, activation='softmax'),
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.layers[0].trainable = True
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

## 模型训练与预测

In [ ]:
epochs = 20
history = model.fit_generator(train_generator,
                              steps_per_epoch = train_num // batch_size,
                              epochs = epochs,
                              validation_data = valid_generator,
                              validation_steps = valid_num // batch_size)

In [ ]:
print(history.history)
def plot_learning_curves(history, label, epcohs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 0, 2)

In [ ]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input)
test_generator = valid_datagen.flow_from_dataframe(
    test_df,
    directory = './',
    x_col = 'filepath',
    y_col = 'class',
    classes = class_names,
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    class_mode = "sparse")
test_num = test_generator.samples
print(test_num)

In [ ]:
# 正式预测
test_predict = model.predict_generator(test_generator,
                                       workers = 3,
                                       use_multiprocessing = True)

In [ ]:
print(test_predict.shape)

In [ ]:
print(test_predict[0:5])

## 预测结果提交

In [ ]:
test_predict_class_indices = np.argmax(test_predict, axis = 1)

In [ ]:
print(test_predict_class_indices[0:5])

In [ ]:
test_predict_class = [class_names[index] 
                      for index in test_predict_class_indices]

In [ ]:
print(test_predict_class[0:5])

In [ ]:
# 产生最终的报告
def generate_submissions(filename, predict_class):
    with open(filename, 'w') as f:
        f.write('id,label\n')
        for i in range(len(predict_class)):
            f.write('%d,%s\n' % (i+1, predict_class[i]))

output_file = "../cifar10/submission_resnet50.csv"
generate_submissions(output_file, test_predict_class)